# Dependencies

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'sklearn'

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aryan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

# Data Processing

In [8]:
# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')


In [9]:
# checkin gthe number of rows and columns
twitter_data.shape

(1599999, 6)

In [10]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [12]:
# naming the columns and reading the dataset again

column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')
twitter_data.columns = column_names
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [13]:
# checkin gthe number of rows and columns
twitter_data.shape

(1599999, 6)

In [14]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [15]:
# counting the number of missing values in the dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [16]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
4    800000
0    799999
Name: count, dtype: int64

# Target Conversion '4' to '1'

In [17]:
twitter_data.replace({'target': {4: 1}}, inplace=True)

In [18]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
1    800000
0    799999
Name: count, dtype: int64

0 -> -ve
1 -> +ve

# Stemming

In [19]:
port_stem = PorterStemmer()

In [20]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [9]:
print(twitter_data.columns)

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')


In [10]:
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', 
                           names=column_names, 
                           encoding='ISO-8859-1', 
                           header=None)


In [11]:
import spacy
import pandas as pd

# Load SpaCy model disabling unnecessary pipeline components for speed
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

# Assuming twitter_data is your DataFrame with the 'text' column
# Use pandas apply to lemmatize
twitter_data['lemmatized_content'] = twitter_data['text'].apply(lemmatize_text)


In [12]:
twitter_data.head()

,target,id,date,flag,user,text,lemmatized_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoot http://twitpic.com/2y1zl - Awww , ..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,be upset that he can not update his Facebook b...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@Kenichan I dive many time for the ball . mana...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feel itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclass no , it be not behave at all ..."


In [16]:
twitter_data.to_csv('twitter_data_lemmatized.csv', index=False)


In [18]:
print(twitter_data[['text', 'lemmatized_content']])


                                                      text  \
0        @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1        is upset that he can't update his Facebook by ...   
2        @Kenichan I dived many times for the ball. Man...   
3          my whole body feels itchy and like its on fire    
4        @nationwideclass no, it's not behaving at all....   
...                                                    ...   
1599995  Just woke up. Having no school is the best fee...   
1599996  TheWDB.com - Very cool to hear old Walt interv...   
1599997  Are you ready for your MoJo Makeover? Ask me f...   
1599998  Happy 38th Birthday to my boo of alll time!!! ...   
1599999  happy #charitytuesday @theNSPCC @SparksCharity...   

                                        lemmatized_content  
0        @switchfoot http://twitpic.com/2y1zl - Awww , ...  
1        be upset that he can not update his Facebook b...  
2        @Kenichan I dive many time for the ball . mana...  
3          

In [19]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    4
1599996    4
1599997    4
1599998    4
1599999    4
Name: target, Length: 1600000, dtype: int64


In [21]:
# separating the data and label
X = twitter_data['lemmatized_content'].values
Y = twitter_data['target'].values

In [23]:
print(X)

['@switchfoot http://twitpic.com/2y1zl - Awww , that be a bummer .   you shoulda get David Carr of Third Day to do it . ;D'
 'be upset that he can not update his Facebook by texte it ... and might cry as a result   School today also . blah !'
 '@Kenichan I dive many time for the ball . manage to save 50 %   the rest go out of bound'
 ... 'be you ready for your MoJo Makeover ? ask I for detail'
 'happy 38th birthday to my boo of alll time ! ! ! Tupac Amaru Shakur'
 'happy # charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H']


In [24]:
print(Y)

[0 0 0 ... 4 4 4]


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)


In [30]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [31]:
print(X_train)

['about to watch see iv and drink a lil wine' '@HaterMagazine I be in !'
 'even though its my favourite drink I think its the vodka and coke that wipe my mind all the time   think I m go to have to find a new drink'
 ... 'be eager for Monday afternoon'
 'hope everyone and their mother have a great day !   can not wait to hear what the guy have in store tomorrow !'
 'I love wake up to Folgers . too bad my voice be deep than his .']


In [32]:
print(X_test)

['@mmangen - M do fine ! I have not have much time to chat on Twitter .    Hubby be back for the summer & amp ; tend to dominate my free time .'
 'at AHS may show w/   ruth kim & amp ; geoffrey sanhueza'
 '@ishatara maybe it be only a bay area thang   dammit !' ...
 '@destini41 nevertheless Hooray ! for 4700 member and have a wonderful and safe trip !'
 'not feel too well' '@supersandro ! ! ! ! thank you !']


In [35]:
# converting the textual data to numerical data
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [36]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 15187913 stored elements and shape (1280000, 579047)>
  Coords	Values
  (0, 39801)	0.2565965644675995
  (0, 519307)	0.1262731536312346
  (0, 548901)	0.2659263385857828
  (0, 457857)	0.24113469356584896
  (0, 253810)	0.5185325455779267
  (0, 56592)	0.1587300525481393
  (0, 166443)	0.36740436789195446
  (0, 311950)	0.4112262315422311
  (0, 556104)	0.43724858183959886
  (1, 225826)	0.9687872468178097
  (1, 79145)	0.12555638726636495
  (1, 246815)	0.2137448572943615
  (2, 519307)	0.14462468961624464
  (2, 56592)	0.09089930805714208
  (2, 166443)	0.420799997006513
  (2, 184072)	0.17818180371201336
  (2, 514797)	0.1705611068367034
  (2, 252744)	0.3036340154825015
  (2, 366790)	0.179254395460741
  (2, 190130)	0.26517056136623157
  (2, 513807)	0.28269352077951
  (2, 508032)	0.15217843080854346
  (2, 545267)	0.30125529995792283
  (2, 128888)	0.28559160648019327
  (2, 507632)	0.10770096957893345
  :	:
  (1279998, 520798)	0.23306817692

In [37]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3699820 stored elements and shape (320000, 579047)>
  Coords	Values
  (0, 55263)	0.1587671302421685
  (0, 74368)	0.1500294193410295
  (0, 79145)	0.06224925768639013
  (0, 117134)	0.247907047269321
  (0, 161602)	0.10357122284823375
  (0, 162644)	0.36959248622991353
  (0, 193431)	0.2353806982296868
  (0, 197722)	0.10564991989700599
  (0, 200155)	0.21876629567514932
  (0, 226065)	0.19571747217553312
  (0, 237908)	0.25965184916678324
  (0, 354748)	0.418681105167566
  (0, 364342)	0.16457620925928335
  (0, 366790)	0.09402018481957657
  (0, 383180)	0.0907954692163159
  (0, 391003)	0.11392733521635347
  (0, 492840)	0.203111405335154
  (0, 505395)	0.32236973677994013
  (0, 508032)	0.07981865188519743
  (0, 516935)	0.29133102021702606
  (0, 519307)	0.15171332353935246
  (0, 530759)	0.1661209232519489
  (1, 45781)	0.531330250631358
  (1, 55263)	0.1823000180202772
  (1, 68630)	0.14653346622786206
  :	:
  (319995, 555307)	0.1601749944274

# Training the ML Model

Logistic Regression

In [39]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)


In [40]:
model.fit(X_train, Y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


# Model Evaluation

Accuracy Score

In [42]:
# accuracy score on the training data
from sklearn.metrics import accuracy_score

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [43]:
print('Accuracy score on the training data : ', training_data_accuracy)

Accuracy score on the training data :  0.8173984375


In [44]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [45]:
print('Accuracy score on the training data : ', test_data_accuracy)

Accuracy score on the training data :  0.797578125


Saving Model

In [46]:
import pickle

In [47]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using saved model

In [ ]:
# loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [48]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
    print('Negative Tweet')
else:
    print('Positive Tweet')

4
[4]
Positive Tweet


In [ ]:
X_new = X_test[3]
print(Y_test[3])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
    print('Negative Tweet')
else:
    print('Positive Tweet')

0
[0]
Negative Tweet


In [5]:
import pandas as pd

# Provide the correct path to your dataset file
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', 
                           names=['target', 'id', 'date', 'flag', 'user', 'text'],
                           encoding='ISO-8859-1', header=None)

# Assuming you already have 'lemmatized_content' or you create it before using fit_transform
# For example, if you do not have 'lemmatized_content', start with 'text'


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_transformed = vectorizer.fit_transform(twitter_data['text'])


In [8]:
import joblib
joblib.dump(vectorizer, 'vectorizer.sav')


['vectorizer.sav']

In [3]:
import pandas as pd

# Load the preprocessed CSV; adjust path if needed
twitter_data = pd.read_csv('twitter_data_lemmatized.csv')

# Check available columns to confirm
print(twitter_data.columns)


Index(['target', 'id', 'date', 'flag', 'user', 'text', 'lemmatized_content'], dtype='object')


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(twitter_data['lemmatized_content'])

model = LogisticRegression(max_iter=1000)
model.fit(X_train, twitter_data['target'])

joblib.dump(vectorizer, 'vectorizer.sav')
joblib.dump(model, 'trained_model.sav')


['trained_model.sav']